In [2]:
import numpy as np
import time as T
from PIL import Image
from cresize import cresizeANN, cresizeABL, cresizeABC, cresizeFNN, cresizeFBL, Test
from functools import lru_cache

FBIT = 13
VBIT = 1 << FBIT
HBIT = 1 << (FBIT - 1)

SBIT = 13
vSBIT = 1 << SBIT
hSBIT = 1 << (SBIT - 1)

def CR(x, y):
    return ((x << FBIT)//y)

def SCR(x, y):
    return ((x << FBIT)//y)

def SDR(x):
    return (x + hSBIT) >> SBIT

XINV = np.array([[-1.,  3., -2.,  0.],
                 [ 3., -6., -3.,  6.],
                 [-3.,  3.,  6.,  0.],
                 [ 1.,  0., -1.,  0.]])/6
YINV = XINV.T

@lru_cache(20)
def clip(x, up, low):
    x = up if (x > up) else x
    x = low if (x < low) else x
    return x

@lru_cache(20)
def reflect(x, up, low):
    x = -x if (x < low) else x
    x = (up+up-x) if (x > up) else x
    return x


In [4]:
# Testing for matrix dot
h1,w1 = 3,4
h2,w2 = 4,5
a = np.random.randint(0,255,(h1,w1))
b = np.random.randint(0,255,(h2,w2))
z = np.empty((h1,w2),dtype=np.int32)
Test(a,b,z,0)
print(h1*w1*h2*w2/1000000)

0.00024


In [5]:
print(z)
print(a@b)

[[ 60278 117195 133661  65620 114943]
 [ 18365 123111 115045  48092  92430]
 [ 44636 125198 128230  40054  95812]]
[[ 60278 117195 133661  65620 114943]
 [ 18365 123111 115045  48092  92430]
 [ 44636 125198 128230  40054  95812]]


In [7]:
RUN = 10000
import time
ti = time.time()
for i in range(RUN):
    #a = np.random.randint(0,255,(h1,w1))
    #b = np.random.randint(0,255,(h2,w2))
    z = np.empty((h1,w2),dtype=np.int32)
    Test(a,b,z,0)
toc1 = time.time() - ti
ti = time.time()
for i in range(RUN):
    #a = np.random.randint(0,255,(h1,w1))
    #b = np.random.randint(0,255,(h2,w2))
    c = a@b
toc2 = time.time() - ti
print(toc1, toc2)

0.0781259536743164 0.02212834358215332


## Resize for nearest neighbor

In [8]:
W, H, C = 400, 300, 3
im = np.arange( W * H * C, dtype=np.uint8).reshape(C, H, W).transpose(1, 2, 0)
rW, rH, rC = 123, 200, 3

In [9]:
def CLIP(val, lower, upper):
    val = lower if val < lower else val
    val = upper if val > upper else val
    return val

In [10]:
def resizeNN(im, w, h):
    ih, iw, ic = im.shape
    w_r = iw/w; h_r = ih/h
    img = np.empty((h, w, ic), dtype=np.uint8)
    p =0
    for i in range(h):
        hi = int(i*h_r + 0.5)
        for j in range(w):
            wi = int(j*w_r + 0.5)
            for k in range(ic):
                img[i,j,k] = im[hi, wi, k]
    return img

def resizeANN(im, w, h):
    ih, iw, ic = im.shape
    w_r = CR(iw,w); h_r = CR(ih,h)
    img = np.empty((h, w, ic), dtype=np.uint8)
    p =0
    for i in range(h):
        hi = (i * h_r + HBIT) >> FBIT
        for j in range(w):
            wi = (j * w_r + HBIT) >> FBIT 
            for k in range(ic):
                img[i,j,k] = im[hi, wi, k]
    return img



In [11]:
dog = Image.open("Dog.jpg")
dog.show()

In [12]:
rdog = resizeANN(np.array(dog), rH, rW)
im = Image.fromarray(rdog)
im.show()

In [25]:
## Time benchmark for floating point calc and fixed point calc

In [13]:
import time as T

RUN = 50
W,H = 224, 224
ti = T.time()
for i in range(RUN):
    rdog = resizeANN(np.array(dog), W, H)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = resizeNN(np.array(dog), W, H)
toc2 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = np.asarray(cresizeANN(np.array(dog), W, H))
toc3 = T.time() - ti
print("toc1 vs toc2 vs toc3", toc1, toc2, toc3)

toc1 vs toc2 vs toc3 4.758841037750244 5.012707471847534 0.23832225799560547


In [8]:
"""
https://stackoverflow.com/questions/12729228/simple-efficient-bilinear-interpolation-of-images-in-numpy-and-python
"""

'\nhttps://stackoverflow.com/questions/12729228/simple-efficient-bilinear-interpolation-of-images-in-numpy-and-python\n'

## resize bilinear interpolation

In [14]:
def resizeBL(im, w, h):
    im_h, im_w, im_c = im.shape
    h_r = im_h / h
    w_r = im_w / w
    img = np.empty((h,w,im_c))
    
    for j in range(h):
        h0 = j * h_r
        n_h0 = int(h0)
        n_h1 = n_h0 + 1
        n_h1 = np.clip(n_h1, 0, im_h-1)
        h0 = h0 % 1
        h0V = 1-h0
        for k in range(w):
            w0 = k * w_r
            n_w0 = int(w0)
            n_w1 = n_w0 + 1
            n_w1 = np.clip(n_w1, 0, im_w-1)
            w0 = w0 % 1
            w0V = 1 - w0
            for i in range(im_c):
                #pix00 = im[n_h0, n_w0, i]
                #pix01 = im[n_h0, n_w1, i]
                #pix10 = im[n_h1, n_w0, i]
                #pix11 = im[n_h1, n_w1, i]
                #print("w0:%.2f, h0:%.2f, n_h0:%d, n_w0:%d, h_r:%.2f, w_r:%.2f" % (w0, h0, n_h0, n_w0, h_r, w_r))
                row_pix0 = im[n_h0, n_w0, i] * w0 + im[n_h0, n_w1, i] * w0V               
                row_pix1 = im[n_h1, n_w0, i] * w0 + im[n_h1, n_w1, i] * w0V

                img[j, k, i] = int(row_pix0 * h0 + row_pix1 * h0V)
                img[j, k, i] = CLIP(img[j, k, i], 0 ,255)

    return img.astype(np.uint8)             

def resizeABL(im, w, h):
    im_h, im_w, im_c = im.shape
    h_r = CR(im_h, h)
    w_r = CR(im_w, w)
    img = np.empty((h,w,im_c))

    for j in range(h):
        h0 = j * h_r
        n_h0 = h0 >> FBIT 
        n_h1 = n_h0 + 1
        #n_h1 = CLIP(n_h1, 0, im_h-1)
        h0 = (h0 + HBIT) % VBIT
        h0V = VBIT-h0
        for k in range(w):
            w0 = k * w_r
            n_w0 = w0 >> FBIT 
            n_w1 = n_w0 + 1
            #n_w1 = CLIP(n_w1, 0, im_w-1)
            w0 = (w0 + HBIT) % VBIT
            w0V = VBIT-w0
            for i in range(im_c):
                #pix00 = im[n_h0, n_w0, i]
                #pix01 = im[n_h0, n_w1, i]
                #pix10 = im[n_h1, n_w0, i]
                #pix11 = im[n_h1, n_w1, i]
                row_pix0 = ((im[n_h0, n_w0, i] * w0 + im[n_h0, n_w1, i] * w0V) + HBIT) >> FBIT
                row_pix1 = ((im[n_h1, n_w0, i] * w0 + im[n_h1, n_w1, i] * w0V) + HBIT) >> FBIT

                img[j, k, i] = ((row_pix0 * h0 + row_pix1 * h0V) + HBIT) >> FBIT
                #img[j, k, i] = CLIP(img[j, k, i], 0, 255)
    return img.astype(np.uint8)             

In [15]:
dog = Image.open("Dog.jpg")
dog_array = np.array(dog)


In [16]:
dog_bi = resizeBL(dog_array, 200, 144)

In [18]:
ndog = Image.fromarray(dog_bi)
ndog.show()

In [19]:
dog_bi = resizeABL(dog_array, 200, 144)

In [20]:
ndog = Image.fromarray(dog_bi)
ndog.show()

In [15]:
import time as T

RUN = 10
W,H = 224, 224
ti = T.time()
for i in range(RUN):
    rdog = resizeBL(np.array(dog), W, H)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = resizeABL(np.array(dog), W, H)
toc2 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cresizeABL(np.array(dog), W, H)
toc3 = T.time() - ti

print("toc1 vs toc2 vs toc3", toc1, toc2, toc3)


toc1 vs toc2 vs toc3 28.619637727737427 27.005561590194702 0.0499720573425293


In [22]:
## Bicubic

In [22]:
def resizeBC(im, h, w):
    """
    @brief 
    Sample code:
    https://stackoverflow.com/questions/52700878/bicubic-interpolation-python
    
    BiCubic Interpolation explain:
    http://www.ahinson.com/algorithms_general/Sections/InterpolationRegression/InterpolationBicubic.pdf
    @param im = src image
    @param h = dst height
    @param w = dst width
    @retval img = resize 2d array with shape = (h, w)
    """
    ih, iw, ic = im.shape
    ih_, iw_ = ih-1, iw-1
    img = np.empty((h, w, ic),dtype=np.uint8)
    h_r = ih / h
    w_r = iw / w
    deltax = 1 #xi[1] - xi[0]
    deltay = 1 #yi[1] - yi[0]
    
    f = np.empty((4,4), np.uint8)
    met = clip
    
    for i in range(ic):
        for j in range(h):
            for k in range(w):

                #if xi.min() <= x <= xi.max() and yi.min() <= y <= yi.max():
                
                # common 
                x = k * w_r
                y = j * h_r
                W = int(x)
                H = int(y)
                
                # pad zero for out of bound value
                px = x - W
                py = y - H
                
                #px = (x-x1)/(x2-x1)
                #py = (y-y1)/(y2-y1)

                #f00 = zi[i-1, j-1]      #row0 col0 >> x0,y0
                #f01 = zi[i-1, j]        #row0 col1 >> x1,y0
                #f02 = zi[i-1, j+1]      #row0 col2 >> x2,y0
                """
                if (H == 0) and (W == 0):
                    f[0,0] = im[H  , W  , i]
                elif (W == 0):
                    f[0,0] = im[H-1  , W  , i]
                    f[1,0] = im[H    , W  , i]
                    f[2,0] = im[H+1  , W  , i]
                    f[3,0] = im[H+2  , W  , i]
                elif (H == 0):
                    f[0,0] = im[H  , W-1, i]      #row0 col0 >> x0,y0
                    f[0,1] = im[H  , W  , i]        #row0 col1 >> x1,y0
                    f[0,2] = im[H  , W+1, i]      #row0 col2 >> x2,y0
                    f[0,3] = im[H  , W+2, i]
                else:
                    f[0,0] = im[H-1, W-1, i]      #row0 col0 >> x0,y0
                    f[0,1] = im[H-1, W  , i]        #row0 col1 >> x1,y0
                    f[0,2] = im[H-1, W+1, i]      #row0 col2 >> x2,y0
                    f[0,3] = im[H-1, W+2, i]
                """
                    
                #f10 = zi[i, j-1]        #row1 col0 >> x0,y1
                #f11 = p00 = zi[i, j]    #row1 col1 >> x1,y1
                #f12 = p01 = zi[i, j+1]  #row1 col2 >> x2,y1
                d0w = met(W-1, iw_, 0)
                d1w = met(W  , iw_, 0)
                d2w = met(W+1, iw_, 0)
                d3w = met(W+2, iw_, 0)   
                #print((j,k,i),d0w,d1w,d2w,d3w,(ih,iw,ic),'size_r',(h_r,w_r),(j*h_r,k*w_r),met(W-1, iw_, 0))
                for q in range(4):
                    d0h = met(H+q-1, ih_, 0)
                    f[q,0] = im[d0h, d0w, i]      
                    f[q,1] = im[d0h, d1w, i]       
                    f[q,2] = im[d0h, d2w, i]     
                    f[q,3] = im[d0h, d3w, i] 
                f = f
                #print(f.shape,XINV.shape, XINV, type(f), type(XINV), f.dtype, XINV.dtype)
                Cr = f@XINV
                PPX = np.array([px*px*px, px*px, px, 1])
                #print('px:',px,'\n',PPX,'\n')
                #print("CR PPX",Cr.shape, PPX.shape, type(Cr), type(PPX), Cr.dtype, PPX.dtype)
                R  = Cr@PPX
                Cc = YINV@R

                img[j,k,i]= clip(int(Cc@(np.array([py*py*py, py*py, py, 1]))),255,0)

    return img

def npdot(x, y, h1, w1, h2, w2):
    assert w1 == h2, 'not valid matmul'
    if h2 == 0: 
        h2 = w2
    out = np.zeros((h1,w2))
    for i in range(h1):
        for j in range(w2):
            for k in range(w1):
                out[i,j] += x[i,k] * y[k,j]
    return out
    
def resizeFBC(im, h, w):
    """
    @brief 
    Sample code:
    https://stackoverflow.com/questions/52700878/bicubic-interpolation-python
    
    BiCubic Interpolation explain:
    http://www.ahinson.com/algorithms_general/Sections/InterpolationRegression/InterpolationBicubic.pdf
    @param im = src image
    @param h = dst height
    @param w = dst width
    @retval img = resize 2d array with shape = (h, w)
    """
    ih, iw, ic = im.shape
    ih_, iw_ = ih-1, iw-1
    img = np.empty((h, w, ic),dtype=np.uint8)
    h_r = ih / h
    w_r = iw / w
    deltax = 1 #xi[1] - xi[0]
    deltay = 1 #yi[1] - yi[0]
    
    f = np.empty((4,4), np.uint8)
    met = clip
    for j in range(h):
        y = j * h_r
        H = int(y)
        py = y - H
        pyy = py*py
        pyyy = pyy * py
        PPY = np.array([pyyy, pyy, py, 1])
        for k in range(w):
            x = k * w_r
            W = int(x)
            px = x - W
            pxx = px*px
            pxxx = pxx * px
            PPX = np.array([pxxx, pxx, px, 1])
            d0w = met(W-1, iw_, 0)
            d1w = met(W  , iw_, 0)
            d2w = met(W+1, iw_, 0)
            d3w = met(W+2, iw_, 0)   
            for i in range(ic):
                for q in range(4):
                    d0h = met(H+q-1, ih_, 0)
                    f[q,0] = im[d0h, d0w, i]      
                    f[q,1] = im[d0h, d1w, i]       
                    f[q,2] = im[d0h, d2w, i]     
                    f[q,3] = im[d0h, d3w, i] 
                f = f
                Cr = f@XINV
                R  = Cr@PPX
                Cc = YINV@R

                img[j,k,i]= clip(int(Cc@PPY),255,0)

    return img


def resizeABC(im, h, w):
    """
    @brief 
    Sample code:
    https://stackoverflow.com/questions/52700878/bicubic-interpolation-python
    
    BiCubic Interpolation explain:
    http://www.ahinson.com/algorithms_general/Sections/InterpolationRegression/InterpolationBicubic.pdf
    @param im = src image
    @param h = dst height
    @param w = dst width
    @retval img = resize 2d array with shape = (h, w)
    """
    ih, iw, ic = im.shape
    ih_, iw_ = ih-1, iw-1
    img = np.empty((h, w, ic),dtype=np.uint8)
    F = np.empty((4,4), dtype=np.uint8)
    h_r = SCR(ih, h)
    w_r = SCR(iw, w)
   
    met = clip
    
    for j in range(h):
        y = j * h_r
        _H = y >> SBIT
        H = _H << SBIT
        py = y - H
        pyy =  SDR(py*py)
        pyyy = SDR(pyy * py)
        PPY = np.array([pyyy, pyy, py, vSBIT])
        for k in range(w):
            x = k * w_r
            _W = x >> SBIT
            W = _W << SBIT
            px = x - W
            pxx = SDR(px * px)
            pxxx  = SDR(pxx * px)
            PPX = np.array([pxxx, pxx, px, vSBIT])
            
            d0w = met(_W-1, iw_, 0)
            d1w = met(_W  , iw_, 0)
            d2w = met(_W+1, iw_, 0)
            d3w = met(_W+2, iw_, 0) 
            for i in range(ic):  
                for q in range(4):
                    d0h = met(_H+q-1, ih_, 0)
                    F[q,0] = im[d0h, d0w, i]      
                    F[q,1] = im[d0h, d1w, i]       
                    F[q,2] = im[d0h, d2w, i]     
                    F[q,3] = im[d0h, d3w, i] 
                
                Cr = F@XINV
                #print('px:',px,'\n',PPX,'\n')
                #print("CR PPX",Cr.shape, PPX.shape, type(Cr), type(PPX), Cr.dtype, PPX.dtype)
                R  = Cr@PPX 
                Cc = (YINV@R + hSBIT) / vSBIT
                pix = Cc@PPY
                img[j,k,i]= clip((Cc@PPY)/vSBIT,255,0)
                #print((j,k,i),(img[j,k,i],pix))

    return img.astype(np.uint8)


In [24]:
import cv2

In [26]:
W,H = 224,224
RUN = 1
ti = T.time()
for i in range(RUN):
    rdoga = resizeBC(dog_array, H, W)
toc1 = T.time() - ti

ti = T.time()
for i in range(RUN):
     rdogb = resizeFBC(dog_array, H, W)
toc2 = T.time() - ti

ti = T.time()
for i in range(RUN):
     rdogc = resizeABC(dog_array, H, W)
toc3 = T.time() - ti

ti = T.time()
for i in range(RUN):
     rdogd = cv2.resize(dog_array, (W, H), interpolation=cv2.INTER_CUBIC)
toc4 = T.time() - ti

print(toc1, toc2, toc3, toc4)

4.066093444824219 2.823791742324829 4.316901922225952 0.0


In [27]:
stacks = np.vstack([rdoga,rdogb,rdogc, rdogd])
imb = Image.fromarray(stacks)
imb.show()

## Performance Testing
### CV2 vs CYTHON resize

In [28]:
dog = Image.open("Dog.jpg")
dog_array = np.array(dog)


In [29]:
import cv2

In [32]:
RUN = 1000
W,H = 224, 224
ti = T.time()
for i in range(RUN):
    rdognn = cv2.resize(dog_array, (W, H), interpolation=cv2.INTER_NEAREST)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdogbl = cv2.resize(dog_array, (W, H), interpolation=cv2.INTER_LINEAR)
toc2 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(dog_array, (W, H), interpolation=cv2.INTER_AREA)
toc3 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdogbc = cv2.resize(dog_array, (W, H), interpolation=cv2.INTER_CUBIC)
toc4 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(dog_array, (W, H), interpolation=cv2.INTER_LANCZOS4)
toc5 = T.time() - ti
print("toc1:%.4f\ntoc2:%.4f\ntoc3:%.4f\ntoc4:%.4f\ntoc5:%f" %( toc1, toc2, toc3, toc4, toc5))
ti = T.time()
for i in range(RUN):
    rdogann = np.asarray(cresizeANN(dog_array, H, W))
toc6 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdogabl = np.asarray(cresizeABL(dog_array, H, W))
toc7 = T.time() - ti
print("toc6c:%.4f\ntoc7c:%.4f\n" % (toc6, toc7))
ti = T.time()

for i in range(RUN):
    rdogfnn = np.asarray(cresizeFNN(dog_array, H, W))
toc9 = T.time() - ti
for i in range(RUN):
    rdogfbl = np.asarray(cresizeFBL(dog_array, H, W))
toc10 = T.time() - ti
print("toc9c:%.4f\ntoc10c:%.4f" % (toc9, toc10))


toc1:0.1315
toc2:0.5170
toc3:3.0769
toc4:0.3623
toc5:2.480326
toc6c:0.2811
toc7c:0.9024

toc9c:0.2005
toc10c:1.2031


In [12]:
np.vstack

<function numpy.vstack(tup)>

In [33]:
stacks = np.hstack([rdogabl, rdogbl])
vstacks = np.hstack([rdogfbl, rdogbl])
v1stacks = np.hstack([rdogann,rdognn])
v2stacks = np.hstack([rdogfnn,rdogfnn])
stacks = np.vstack([stacks,vstacks,v1stacks,v2stacks])
img = Image.fromarray(stacks)
img.show()
